In [1]:
import sys
import os
sys.path.append(os.path.abspath("Pytorch/"))
sys.path.append(os.path.abspath("models/"))

In [2]:
from XVA import *
from XVAFBSNNs import *

In [3]:
from FBSNNs import *
from CallOptionsBasket import *

In [4]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 100  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [D + 1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
model = CallOptionsBasket(Xi, T, M, N, D, Mm, layers, mode, activation)

n_iter = 2 * 10**4
lr = 1e-3

In [5]:
model.load_model("models/100DCallOptionXVAPaper.pth")

In [8]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

M = 64  # number of trajectories (batch size)
N = 100  # number of time snapshots
D = 100 # number of dimensions
Mm = N ** (1/5)

layers = [1+1] + 4 * [256] + [1]

Xi = np.array([1] * int(D))[None, :]
T = 1.0

"Available architectures"
mode = "NAIS-Net"  # FC, Resnet and NAIS-Net are available
activation = "Sine"  # Sine and ReLU are available
xvamodel = XVA(Xi, T, M, N, D, Mm, layers, mode, activation, model)

n_iter = 2 * 10**3
lr = 1e-3

In [10]:
n_iter = 1 * 10**3
lr = 1e-5

In [11]:
tot = time.time()
print(xvamodel.device)
graph = xvamodel.train(n_iter, lr)
print("total time:", time.time() - tot, "s")

cuda:0
It: 1900, Loss: 2.404e-03, Y0: 0.089, Time: 0.85, Learning Rate: 1.000e-05
It: 2000, Loss: 3.407e-06, Y0: 0.040, Time: 89.17, Learning Rate: 1.000e-05
It: 2100, Loss: 8.857e-06, Y0: 0.041, Time: 90.38, Learning Rate: 1.000e-05
It: 2200, Loss: 1.434e-05, Y0: 0.039, Time: 90.23, Learning Rate: 1.000e-05
It: 2300, Loss: 6.412e-06, Y0: 0.041, Time: 93.11, Learning Rate: 1.000e-05
It: 2400, Loss: 3.386e-06, Y0: 0.043, Time: 89.65, Learning Rate: 1.000e-05
It: 2500, Loss: 1.476e-05, Y0: 0.042, Time: 89.03, Learning Rate: 1.000e-05
It: 2600, Loss: 2.874e-05, Y0: 0.042, Time: 90.39, Learning Rate: 1.000e-05
It: 2700, Loss: 1.732e-05, Y0: 0.043, Time: 89.07, Learning Rate: 1.000e-05
It: 2800, Loss: 1.188e-05, Y0: 0.045, Time: 89.68, Learning Rate: 1.000e-05
total time: 899.7136900424957 s


In [17]:
t_test, W_test, C_test = xvamodel.fetch_minibatch()
X_pred, Y_pred = xvamodel.predict(C_test, t_test, W_test)

if type(t_test).__module__ != 'numpy':
    t_test = t_test.cpu().numpy()
if type(X_pred).__module__ != 'numpy':
    X_pred = X_pred.cpu().detach().numpy()
if type(Y_pred).__module__ != 'numpy':
    Y_pred = Y_pred.cpu().detach().numpy()

for i in range(64):
    t_test_i, W_test_i, C_test_i = xvamodel.fetch_minibatch()
    X_pred_i, Y_pred_i = xvamodel.predict(C_test_i, t_test_i, W_test_i)
    if type(X_pred_i).__module__ != 'numpy':
        X_pred_i = X_pred_i.cpu().detach().numpy()
    if type(Y_pred_i).__module__ != 'numpy':
        Y_pred_i = Y_pred_i.cpu().detach().numpy()
    X_pred = np.concatenate((X_pred, X_pred_i), axis=0)
    Y_pred = np.concatenate((Y_pred, Y_pred_i), axis=0)

In [18]:
Y_pred.shape

(65, 101, 1)

In [19]:
Y_preds = Y_pred.reshape(65, 101)

In [20]:
pred_fva = Y_preds[:,0][0]

In [22]:
from scipy.stats import multivariate_normal as normal
dw_sample = normal.rvs(size=[40000, D, N]) * np.sqrt(T/N)
x_sample = np.zeros([40000, D, N + 1]) 
x_sample[:, :, 0] = np.ones([40000, D]) * 1
factor = np.exp((0.01-(0.25**2)/2)*(T/N))
for i in range(N):   
    x_sample[:, :, i + 1] = (factor * np.exp(0.25 * dw_sample[:, :, i])) * x_sample[:, :, i]

In [24]:
portfolio = np.sum(x_sample, axis=1, keepdims=True)
payoff = np.maximum(portfolio - 100 * 1, 0)
average = np.mean(payoff, axis=0, keepdims=True)
mc_price = np.exp(-0.01 * 1) * average[:, :, 100]

In [25]:
mc_price[0][0]

1.5949219069576714

In [26]:
mc_price_rf = np.exp(-0.04 * 1) * average[:, :, 100]

In [27]:
mc_price_rf[0][0]

1.5477848409657415

In [28]:
exact_fva = mc_price[0][0] - mc_price_rf[0][0]

In [29]:
pred_fva

0.04431065

In [30]:
exact_fva

0.047137065991929905

In [31]:
(pred_fva - exact_fva) ** 2 / exact_fva ** 2

0.0035953944316089066

In [32]:
xvamodel.save_model("models/100DCallOptionFVAXVAPaper.pth")